In [74]:
import pandas as pd
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import RFE
from sklearn.feature_selection import f_classif,SelectPercentile
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.metrics import classification_report 



In [47]:
data_no_additional_feature = 'training_users.csv'
data_with_tweet_csv = 'training_user_tweet.csv'
data_with_lsa = ''
#from data import online_features
import numpy as np
"""
  Remove the field with objects
"""
def remap_fields(df):

    for name, dtype in zip(list(df), df.dtypes):
        
        if dtype == 'object':
            df[name] = df[name].map( lambda x: 1 if  x else 0)
    df.fillna(0, inplace = True)
    return df
            


"""
    get unified function get dataset
"""

def get_dataset(data_type='none'):

    if data_type == 'none':
        data = pd.read_csv(data_no_additional_feature)
        data = remap_fields(data)
        train_x, test_x, _, _ = train_test_split(data, data.label,  stratify =data.label)
        return train_x, test_x
    elif data_type == 'lda':
        data = pd.read_csv(data_with_tweet_csv)
        train_x, test_x, _, _ = train_test_split(data, data.label,  stratify =data.label)

        train_x , test_x= topic_model(train_x, test_x)
        train_x = remap_fields(train_x)
        test_x = remap_fields(test_x)
        return train_x, test_x

In [48]:
data = pd.read_csv(data_with_tweet_csv)

In [63]:
import pymongo, requests, json, urllib, re, collections
from pymongo import MongoClient
from collections import OrderedDict
import math

client = MongoClient('mongodb://localhost:27017/')
#mongodb database and collection
db = client.dataset_agnezmo
dbPre = db.preprocessing

class spellCheck:
    #Peter Norvig Spelling Correction Algorithm based on Bayes' Theorem
    #http://norvig.com/spell-correct.html
    def train(self,features):
        model = collections.defaultdict(lambda: 1)
        for f in features:
            model[f] += 1
        return model
    
    def __init__(self):
        self.NWORDS = self.train(self.words(open('id_dict/spellingset.txt').read()))
        self.alphabet = 'abcdefghijklmnopqrstuvwxyz'
        
    def words(self,text): return re.findall('[a-z]+', text.lower()) 
    
    def edits1(self, word):
        splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes    = [a + b[1:] for a, b in splits if b]
        transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
        replaces   = [a + c + b[1:] for a, b in splits for c in self.alphabet if b]
        inserts    = [a + c + b     for a, b in splits for c in self.alphabet]
        return set(deletes + transposes + replaces + inserts)

    def known_edits2(self, word):
        return set(e2 for e1 in self.edits1(word) for e2 in self.edits1(e1) if e2 in self.NWORDS)

    def known(self,words): return set(w for w in words if w in self.NWORDS)

    def correct(self, word):
        candidates = self.known([word]) or self.known(self.edits1(word)) or self.known_edits2(word) or [word]
        return max(candidates, key=self.NWORDS.get)

class sentiStrength:
    
    def __init__(self):
        self.__sentiDict = [line.replace('\n','') for line in open('id_dict/sentimentword.txt').read().splitlines()]
        self.__emotDict = [line.replace('\n','') for line in open("id_dict/emoticon.txt").read().splitlines()]
        self.__negatingDict = [line.replace('\n','') for line in open("id_dict/negatingword.txt").read().splitlines()]
        self.__boosterDict = [line.replace('\n','') for line in open("id_dict/boosterword.txt").read().splitlines()]
        self.__idiomDict = [line.replace('\n','') for line in open("id_dict/idiom.txt").read().splitlines()]
        self.__questionDict = [line.replace('\n','') for line in open("id_dict/questionword.txt").read().splitlines()]
        self.__katadasar = [line.replace('\n','') for line in open('id_dict/rootword.txt').read().splitlines()]
        self.p = 0
        self.n = 0
        self.nn = 0
        
    def correctSpelling(self,text):
        sc = spellCheck()
        return sc.correct(text) if text not in self.__katadasar else text #for t in text.split()
    def createBoosterDict(self):
        scores={}
        for line in self.__boosterDict:
            term, score= line.split()
            scores[term] = int(score)
        return scores
    def createSentiDict(self):
        scores={}
        for line in self.__sentiDict:
            term, score= line.split()
            scores[term] = int(score)
        return scores
    def createEmoticonDict(self):
        scores={}
        for line in self.__emotDict:
            term, score= line.split("\t")
            scores[term] = int(score)
        return scores
    def createIdiomDict(self):
        scores={}
        for line in self.__idiomDict:
            term, score= line.split("\t")
            scores[term] = int(score)
        return scores
    def main(self, text):
        sentimen = self.createSentiDict()
        senti_keys = sentimen.keys()
        
        emoticon = self.createEmoticonDict()
        emo_keys = emoticon.keys()
        
        booster = self.createBoosterDict()
        boost_keys = booster.keys()
        
        idiom = self.createIdiomDict()
        idiom_keys = idiom.keys()
        
        neg = -1
        pos = 1
        
        sentence_score =[]
        questionTerm = []
        
        #if math.isnan(text):
        if pd.isnull(text):
            score=float('-inf')
        else:
            tweet = text.split()

            for term in tweet:
                #print(term)

                score = 0
                term_index = tweet.index(term)
                #spelling correction (optional) - it slows down processing time
                #term = self.correctSpelling(term)            
                count_term = len(tweet)
                bigram = "{} {}".format(tweet[term_index-1],term)
                #filter indonesian plural form, ex: keren-keren -> keren
                term = re.sub('(\w+)-(\w+)',r'\1',term)
                if term.isalpha():                
                    if term in senti_keys:
                        score = sentimen[term]
                        #handle negating words
                        if tweet[term_index-1] in self.__negatingDict:
                            score = -abs(score) if score>0 else abs(score)
                        #handle boosting words
                        if term_index > 0 and term_index < count_term-1:
                            if tweet[term_index-1] in boost_keys:
                                #print tweet[term_index-1], term
                                score += booster[tweet[term_index-1]]
                            elif tweet[term_index+1] in boost_keys:
                                #print term, tweet[term_index+1]
                                score += booster[tweet[term_index+1]]

                        #increase by 1 consecutive term with minimum score 3 in positive term and -3 in negative term
                        if tweet[term_index-1] in senti_keys:
                            if score >= 3:
                                score+=1
                            elif score<= -3:
                                score-=1
                            else:
                                score
                            #print tweet[term_index-1], term, score

                    #handle idiom
                    if bigram in idiom_keys:
                        score = idiom[bigram] 
                    #handle question
                    if term in self.__questionDict:
                        isQuestion = True
                        questionTerm.append(term)

                else:
                    #handle emoticon
                    if term.encode('utf-8',errors='ignore') in emoticon.keys():                    
                        score = emoticon[term]
                    #check if (?) sign exist
                    elif re.search(r'\?',term):
                        isQuestion = True
                        questionTerm.append(term)

                    #exclamation mark give minimum +2
                    elif re.search('!',term):
                        score = 2

                    #more exclamation mark boost preceeding word by 1, ex: good!!!        
                    elif re.sub('(\w+)[!]+$',r'\1',term) in senti_keys:
                        #print term
                        score = sentimen[term]
                        if score > 0:
                            score +=1
                        elif score <0:
                            score -=1
                    else:
                        score = 0

                #max(positive), max(negative)
                pos= score if score > pos else pos
                neg= score if score < neg else neg

                if score != 0:
                    #insert score between term
                    term = "{} [{}]".format(term, score)
                sentence_score.append(term)

            isQuestion = True if len(questionTerm)>0 else False

            if abs(pos) > abs(neg):               
                self.countSentimen("+")
                senti_result = "result: +positive"
            elif abs(pos) < abs(neg) and not isQuestion:
                self.countSentimen("-")
                senti_result = "result: -negative"
                #ignoring negative sentiment in question as neutral
            elif abs(pos) < abs(neg) and isQuestion:
                self.countSentimen("?")
                senti_result = "result: neutral"
            else:
                self.countSentimen("?")
                senti_result = "result: neutral"

            result = ' '.join(sentence_score)
            result = "{} [score:{},{}][{}]".format(result,neg, pos, senti_result)
            return result
    def countSentimen(self, res):
        if res=="+":
            self.p+=1
        elif res =="-":
            self.n+=1
        else:
            self.nn+=1
        return
    def getSentimenScore(self):
        return [self.p,self.n,self.nn]


In [65]:
sample_tweet_sentiment = pd.DataFrame(columns = ["id", "positive_senti","negative_senti", "neutral_senti"])
def main():
    ss = sentiStrength()
    sc = spellCheck()
    #tweets = data['tweet']
    #users= data['id']
    #print(tweets)
    i=0
    for j in range(len(data)):
        ss.main(data.loc[j,'tweet'])
        user=data.loc[j,'id']
        #print("=====================")        
        #print(ss.getSentimenScore())
        score=ss.getSentimenScore()
        #print(score[0],score[1], score[2])
        sample_tweet_sentiment.loc[i] = [user,score[0],score[1], score[2]]
        i=i+1
    #ss.main(tweets)
    
    
main()

KeyboardInterrupt: 

In [66]:
data_org=pd.read_csv(data_no_additional_feature)
new_df = pd.merge(data_org, sample_tweet_sentiment,  how='inner', left_on=['id',], right_on = ['id'])

In [67]:
new_df

,Unnamed: 0,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,url,...,protected,verified,description,created_at,updated,file,label,positive_senti,negative_senti,neutral_senti
0,0,1502026416,TASUKU HAYAKAWA,0918Bask,2177,208,332,265,1,NaN,...,NaN,NaN,15years ago X.Lines24,Tue Jun 11 11:20:35 +0000 2013,2016-03-15 15:53:47,hum1_us,0,0,0,0
1,1,2492782375,ro_or,1120Roll,2660,330,485,3972,5,NaN,...,NaN,NaN,保守見習い地元大好き人間。 経済学、電工、仏教を勉強中、ちなDeではいかんのか？ (*^◯^*),Tue May 13 10:37:57 +0000 2014,2016-03-15 15:53:48,hum1_us,0,0,0,0
2,2,293212315,bearclaw,14KBBrown,1254,166,177,1185,0,NaN,...,NaN,NaN,Let me see what your best move is!,Wed May 04 23:30:37 +0000 2011,2016-03-15 15:53:48,hum1_us,0,0,0,0
3,3,191839658,pocahontas farida,wadespeters,202968,2248,981,60304,101,http://t.co/rGV0HIJGsu,...,NaN,NaN,20. menna: #farida #nyc and the 80s actually y...,Fri Sep 17 14:02:10 +0000 2010,2016-03-15 15:53:48,hum1_us,0,1,0,0
4,4,3020965143,Ms Kathy,191a5bd05da04dc,82,21,79,5,0,NaN,...,NaN,NaN,Cosmetologist,Fri Feb 06 04:10:49 +0000 2015,2016-03-15 15:53:48,hum1_us,0,1,0,0
5,5,2157382005,JO,19_Joanne_87,12915,641,1066,1568,7,NaN,...,NaN,NaN,CHRISTIAN -Communication degree -graphic desig...,Mon Oct 28 16:08:18 +0000 2013,2016-03-15 15:53:49,hum1_us,0,1,0,1
6,6,1947320929,❄McKayla❄,1Dniallprincess,13676,1042,2000,19012,7,NaN,...,NaN,NaN,"Live, Young, Wild and Free #crazymofo",Tue Oct 08 15:59:30 +0000 2013,2016-03-15 15:53:49,hum1_us,0,1,0,2
7,7,255846106,Giselle,1GisellePizarro,61294,561,118,590,4,https://t.co/v4qQQ0XDjd,...,NaN,NaN,Hey what's up guys? This is Giselle. I'm 21. C...,Tue Feb 22 04:37:34 +0000 2011,2016-03-15 15:53:49,hum1_us,0,1,0,2
8,8,1733095801,Nicole Romany,1Nicoleromany,4854,337,256,1407,4,NaN,...,NaN,NaN,NaN,Thu Sep 05 20:52:02 +0000 2013,2016-03-15 15:53:50,hum1_us,0,1,0,3
9,9,1336097480,Fallen Angel,1_DErika,2408,421,338,2227,5,NaN,...,NaN,NaN,I am not a perfect angel that you think you se...,Mon Apr 08 08:37:42 +0000 2013,2016-03-15 15:53:50,hum1_us,0,1,0,3


In [68]:
trainY = new_df.loc[:,'label']
trainX=new_df[new_df.columns.difference(['label','id', 'Unnamed: 0', 'name', 'screen_name',
       'url', 'lang', 'time_zone', 'location', 'default_profile',
       'default_profile_image', 'geo_enabled', 'profile_image_url',
       'profile_banner_url', 'profile_use_background_image',
       'profile_background_image_url_https', 'profile_text_color',
       'profile_image_url_https', 'profile_sidebar_border_color',
       'profile_background_tile', 'profile_sidebar_fill_color',
       'profile_background_image_url', 'profile_background_color',
       'profile_link_color', 'utc_offset', 'protected', 'verified',
       'description', 'created_at', 'updated', 'file'])]
#trainY=trainY.fillna(trainY.median())



trainX=np.array(trainX)
trainY =np.array(trainY)

In [77]:
rem_features=['label','id', 'Unnamed: 0', 'name', 'screen_name',
       'url', 'lang', 'time_zone', 'location', 'default_profile',
       'default_profile_image', 'geo_enabled', 'profile_image_url',
       'profile_banner_url', 'profile_use_background_image',
       'profile_background_image_url_https', 'profile_text_color',
       'profile_image_url_https', 'profile_sidebar_border_color',
       'profile_background_tile', 'profile_sidebar_fill_color',
       'profile_background_image_url', 'profile_background_color',
       'profile_link_color', 'utc_offset', 'protected', 'verified',
       'description', 'created_at', 'updated', 'file']

cols=['Features']
feature_importances=pd.DataFrame(columns=cols)
feature_importances['Features'] = new_df[new_df.columns.difference(rem_features)].columns
train_auc=[]
test_auc=[]
cv = StratifiedKFold(n_splits=10)
i = 0
for train, test in cv.split(trainX, trainY):
    # The "accuracy" scoring is proportional to the number of correct
    # classifications
    classifier = xgb.XGBClassifier(objective='binary:logistic' , booster='gbtree', eta=0.001,
                               max_depth=20,colsample_bytree=0.7, subsample=0.8,min_child_weight=5,gamma=0,
                                n_estimators=100, 
                                n_jobs=-1, verbose=False)
    #classifier=xgb.XGBClassifier(**params)
    classifier.fit(trainX[train], trainY[train],
                        eval_set=[(trainX[train], trainY[train]), 
                                  (trainX[test], trainY[test])],
                                eval_metric='logloss', verbose=False)
    
    evals_result= classifier.evals_result()
    text='Value_'+str(i)
    i=i+1
    feature_importances[text] = classifier.feature_importances_

    train_auc.append(evals_result['validation_0']['logloss'][99])
    test_auc.append(evals_result['validation_1']['logloss'][99])
    #print(evals_result)
    

predict = classifier.predict(trainX[test])
print(classification_report(trainY[test], predict))
    
mean_auc_train = np.mean(train_auc)
std_auc_train = 1.96*np.std(train_auc)
# 
mean_auc_test = np.mean(test_auc)
std_auc_test = 1.96*np.std(test_auc)
# 
print('Test set auc:' + str(mean_auc_train) + '(CI :'+ str(std_auc_train) + ')')
print('Train set auc:' + str(mean_auc_test) + '(CI :'+ str(std_auc_test) + ')')

# =============================================================================
# feature_importances = pd.DataFrame(classifier.feature_importances_,
#                                index = edviz_df[edviz_df.columns.difference(['disposition','PatientId'])].columns,
#                                 columns=['importance']).sort_values('importance',ascending=False)
# 
# =============================================================================
#xgb.plot_importance(classifier)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        13

avg / total       1.00      1.00      1.00        13

Test set auc:0.0022757(CI :1.7639999999999792e-06)
Train set auc:0.0022757(CI :1.7639999999999792e-06)


/anaconda3/lib/python3.6/site-packages/xgboost/sklearn.py:523: RuntimeWarning: invalid value encountered in true_divide
  return all_features / all_features.sum()
/anaconda3/lib/python3.6/site-packages/xgboost/sklearn.py:523: RuntimeWarning: invalid value encountered in true_divide
  return all_features / all_features.sum()
/anaconda3/lib/python3.6/site-packages/xgboost/sklearn.py:523: RuntimeWarning: invalid value encountered in true_divide
  return all_features / all_features.sum()
/anaconda3/lib/python3.6/site-packages/xgboost/sklearn.py:523: RuntimeWarning: invalid value encountered in true_divide
  return all_features / all_features.sum()
/anaconda3/lib/python3.6/site-packages/xgboost/sklearn.py:523: RuntimeWarning: invalid value encountered in true_divide
  return all_features / all_features.sum()
/anaconda3/lib/python3.6/site-packages/xgboost/sklearn.py:523: RuntimeWarning: invalid value encountered in true_divide
  return all_features / all_features.sum()
/anaconda3/lib/python3

ValueError: Booster.get_score() results in empty